In [5]:
import pandas as pd
import numpy as np

from dsgd.DSClassifierMultiQ import DSClassifierMultiQ

from sklearn.metrics import accuracy_score, confusion_matrix, f1_score


In [6]:
df = pd.read_csv("https://huggingface.co/datasets/furrutiav/sac_nllf/raw/main/train.csv")
df = df.drop(columns=['index', "b1(Y)", "b2(Y)", "b3(Y)", "b4(Y)", "b5(Y)", "b6(Y)", "b7(Y)", "b8(Y)", "b9(Y)", "b10(Y)", "b11(Y)", "b12(Y)", "b13(Y)"])
print("Dataframe shape: ", df.shape)
df.head()
test = pd.read_csv("https://huggingface.co/datasets/furrutiav/sac_nllf/raw/main/test.csv")
test = test.drop(columns=['index', "b1(Y)", "b2(Y)", "b3(Y)", "b4(Y)", "b5(Y)", "b6(Y)", "b7(Y)", "b8(Y)", "b9(Y)", "b10(Y)", "b11(Y)", "b12(Y)", "b13(Y)"])
print("Dataframe shape: ", test.shape)

Dataframe shape:  (1400, 14)
Dataframe shape:  (400, 14)


In [7]:
y = df['label']
X = df.drop(columns=['label'])
y = y.to_numpy()
X = X.to_numpy()    
y_test = test['label']
X_test = test.drop(columns=['label'])
X_test = X_test.to_numpy()
y_test = y_test.to_numpy()

In [ ]:
DSCs = {}
time = {}
accuracys = {}
learning_rates = [0.0001, 0.001, 0.01, 0.1]
batch_sizes = [2000, 3000, 4000]
min_dl = [1e-7, 1e-6, 1e-5,]
num_rules = [3, 5, 7,]
df_results = pd.DataFrame(columns=['lr', 'bs', 'mdl', 'nr', 'accuracy', 'f1_micro', 'f1_macro'])
for lr in learning_rates:
    for bs in batch_sizes:
        for mdl in min_dl:
            for nr in num_rules:
                DSC = DSClassifierMultiQ(2, min_iter=20, max_iter=500, debug_mode=True, lr=lr, batch_size=bs,
                                lossfn="MSE", num_workers=1, min_dloss=mdl, precompute_rules=True)
                print(f"Training DSC with lr={lr}, bs={bs}, mdl={mdl}, nr={nr}")
                losses, epoch, dt = DSC.fit(X, y, add_single_rules=True,
                            single_rules_breaks=nr, add_mult_rules=False,
                                column_names=df.columns[:-1], print_every_epochs=1, print_final_model=False)
                y_pred = DSC.predict(X_test)
                acc = accuracy_score(y_test, y_pred)
                f1_micro = f1_score(y_test, y_pred, average='micro')
                f1_macro = f1_score(y_test, y_pred, average='macro')
                print(f"Accuracy: {acc}, F1 micro: {f1_micro}, F1 macro: {f1_macro}")
                new_row = pd.DataFrame({
                    'lr': [lr], 'bs': [bs], 'mdl': [mdl], 'nr': [nr],
                    'accuracy': [acc], 'f1_micro': [f1_micro], 'f1_macro': [f1_macro]
                })
                df_results = pd.concat([df_results, new_row])
                

df_results.to_csv("results_neg.csv", index=False)

In [10]:
df_results = pd.read_csv('results_neg.csv')
df_results = df_results.sort_values(by='accuracy', ascending=False)
df_results.head(5)

,lr,bs,mdl,nr,accuracy,f1_micro,f1_macro
16,0.0001,3000,1.000000e-05,5,0.6900,0.6900,0.689930
13,0.0001,3000,1.000000e-06,5,0.6800,0.6800,0.679800
9,0.0001,3000,1.000000e-07,3,0.6750,0.6750,0.673400
6,0.0001,2000,1.000000e-05,3,0.6700,0.6700,0.667874
12,0.0001,3000,1.000000e-06,3,0.6675,0.6675,0.665491
